# Deck Selection
Randomly selects decks for players.

In [1]:
import pandas as pd
import func

Players = ["P0","P1"]
MatchType = "Free"

decks_df = pd.read_csv("https://raw.githubusercontent.com/amcclead7336/MTG_GameData/main/Decks.csv")

Rating_Col = func.get_curr_ratings(decks_df.columns)

base_deck = decks_df.sample(1)
base_power = base_deck[Rating_Col].values[0]

if MatchType == "Equal":
    filter_power = abs(decks_df[Rating_Col] - base_power) == 0

elif MatchType == "Fair":
    filter_power = abs(decks_df[Rating_Col] - base_power) <=1

elif MatchType == "Challenging":
    filter_power = abs(decks_df[Rating_Col] - base_power) == 2

elif MatchType == "Crazy":
    filter_power = abs(decks_df[Rating_Col] - base_power) > 2

else:
    filter_power = True

filter_df = decks_df[(filter_power) & (decks_df['ID'] != base_deck["ID"].values[0])]

try:
    other_decks = filter_df.sample(len(Players)-1)
except ValueError:
    print("Not able to find enough decks for everyone. Change match type or reduce players.")
    raise KeyboardInterrupt

rand_selects = pd.concat([base_deck,other_decks])

for i in range(len(Players)):
    print(Players[i]+"'s Deck is: ")
    print("\t* Deck: {0}\n\t* ID: {1}\n\t* Rating: {2}\n".format(rand_selects.iloc[i][1],rand_selects.iloc[i][0],rand_selects.iloc[i][3]))

P0's Deck is: 
	* Deck: Necron Dynasties
	* ID: 20
	* Rating: nan

P1's Deck is: 
	* Deck: Mishra's Burnished Banner
	* ID: 13
	* Rating: 3.0



# Game Play Update Script
Takes data to append new row in GameData.csv

In [2]:
import datetime

Players = ["P0","P1"]
check = input("Have pulled the latest data: ")
if check == '' or check.lower()[0] != "n":
    today1 = datetime.date.today()

    deck_ids = []
    check = input("Do you want to auto pull Deck ID or manual enter [m:A]: ").lower()
    if check == "" or check[0] != "m":
        for deck_id in rand_selects["ID"]:
            deck_ids.append(deck_id)
    else:
        for player in Players:
            deck_ids.append(input(player+"'s Deck: "))

    times=[]
    life_totals = []
    total_cards = []
    results = []
    for player in Players:
        times.append(input(player+"Time: "))
        life_totals.append(int(input(player+"EndLife: ")))
        total_cards.append(int(input(player+"TotalCards: ")))
        results.append(int(input(player+"Win: ")))

    dur = datetime.timedelta()
    dttimes=[]
    for ptime in times:
        if ptime.count(":") == 2:
            h,m,s = ptime.split(":")
        if ptime.count(":") == 1:
            h,m,s = [0] + ptime.split(":")
        d = datetime.timedelta(hours=int(h), minutes=int(m), seconds=int(s))
        dttimes.append(d)
        dur += d
    print(str(dttimes[0]))

    format_time = "%H:%M:%S"

    row = {"Date":today1, "P0Deck":deck_ids[0], "P1Deck":deck_ids[1], "P0EndLife":life_totals[0], "P1EndLife":life_totals[1], "P0Time":str(dttimes[0]), "P1Time":str(dttimes[1]), "TotalTime":str(dur),
    "P0TotalCards":total_cards[0], "P1TotalCards":total_cards[1], "P0Win":results[0], "P1Win":results[1], "P0":0, "P1":1}

    df_gd = pd.read_csv("GameData.csv")
    df_gd = df_gd.append(row,ignore_index=True)
    print(df_gd.tail())
    check = input("Do you aprove change: ")

    if check == '' or check.lower()[0] != "n":
        df_gd.to_csv("GameData.csv",index=False)
        

Have pulled the latest data: 
Do you want to auto pull Deck ID or manual enter [m:A]: m
P0's Deck: 19
P1's Deck: 20
P0Time: 15:01
P0EndLife: -2
P0TotalCards: 12
P0Win: 0
P1Time: 20:32
P1EndLife: 33
P1TotalCards: 30
P1Win: 1
0:15:01
          Date P0Deck P1Deck  P0EndLife  ...  P0Win P1Win P0 P1
9   2023-01-06      3      9         -1  ...      0     1  0  1
10  2023-01-06      6     11         -3  ...      0     1  0  1
11  2023-01-06     16     12         -6  ...      0     1  0  1
12  2023-01-17     19     17         -5  ...      0     1  0  1
13  2023-01-17     19     20         -2  ...      0     1  0  1

[5 rows x 14 columns]
Do you aprove change: 


# Add Deck
Add Deck to Decks.csv

In [ ]:
import pandas as pd

decks_df2 = pd.read_csv("https://raw.githubusercontent.com/amcclead7336/MTG_GameData/main/Decks.csv")

new_deck = {"ID":decks_df2["ID"].max() +1,
            "Deck Title":"",
           "Commander":"",
           "White":0,
           "Blue":0,
           "Black":0,
           "Red":0,
           "Green":0}

new_deck["Deck Title"] = input("Deck Title: ")
new_deck["Commander"] = input("Commander played with deck: ")
colors = input("Deck Colors (wbgrk): ").lower()

color_dict = {"w":"White", "b":"Blue", "g":"Green", "r":"Red", "k":"Black"}
for k, color in color_dict.items():
    if k in colors:
        new_deck[color] = 1


decks_df2 = decks_df2.append(new_deck,ignore_index=True)

print(decks_df2.tail())
check = input("Proceed: ")
if check == '' or check.lower()[0] != "n":
    decks_df2.to_csv("Decks.csv",index=False)


Deck Title: Necron Dynasties
Commander played with deck: Szarekh, the Silent King
Deck Colors (wbgrk): k
    ID           Deck Title                     Commander  Ratings 2022-11-20  \
16  16    Lorehord Legacies      Osgir, the Reconstructor                 2.0   
17  17       Arm for Battle         Wyleth, Soul of Steel                 2.0   
18  18      Spirit Squadron  Millicent, Restless Revenant                 3.0   
19  19  Nature of the Beast      Marath, Will of the Wild                 NaN   
20  20     Necron Dynasties      Szarekh, the Silent King                 NaN   

    White  Blue  Black  Red  Green                Notes  
16      1     0      0    1      0  Upgraded 2022-11-19  
17      1     0      0    1      0                  NaN  
18      1     1      0    0      0                  NaN  
19      1     0      0    1      1                  NaN  
20      0     0      1    0      0                  NaN  
Proceed: 
